In [19]:
# get html and return subject polarity
# Find related videos from other channels to compare topic polarization
import os
import pandas as pd
from scipy import spatial
from nltk.chunk import conlltags2tree
from spacy import displacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import config

key = os.path.join(config.dirs['apis'],'youtube_secret_key.json')
SCOPES = 'https://www.googleapis.com/auth/youtube.force-ssl'
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(key, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def get_videos(service, **kwargs):
    final_results = []
    results = service.search().list(**kwargs).execute()
 
    i = 0
    max_pages = 3
    while results and i < max_pages:
        final_results.extend(results['items'])
        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.search().list(**kwargs).execute()
            i += 1
        else:
            break
 
    return final_results

def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    final_results = []
    for item in results:
        title = item['snippet']['title']
        video_id = item['id']['videoId']
        comments = get_video_comments(service, part='snippet', videoId=video_id, textFormat='plainText')
        final_results.extend([(video_id, title, comment) for comment in comments])
    return final_results

def find_channel_id(service, search_term):
    request = service.channels().list(
    forUsername=search_term,
    part='id,snippet'
    ).execute()
    channel_id = request['items'][0]['id']
    return channel_id

def search_channels(service, search_term):
    request = service.search().list(
        q=search_term,
        part='id,snippet',
        type='channel',
        maxResults='20'
    ).execute()
    
    channel_df = pd.DataFrame(columns=['channel_id','channel'])
    for results in request:
        chan = request['items'][0]['channelTitle']
        c_id = request['items'][0]['channelId']
        channel_df = channel_df.append(pd.Series([chan, c_id],index=channel_df.columns), ignore_index = True)
    
    return channel_df

def get_video_comments(service, **kwargs):
    # results = service.commentThreads().list(
    #     part='snippet',
    #     videoId='h2RzmSAZ4Hc',
    #     textFormat='plainText'
    #     ).execute()
    # results
    comment_df = pd.DataFrame(columns=['author','comment','likes'])
    results = service.commentThreads().list(**kwargs).execute()
 
    while results:
        for item in results['items']:
            author = item['snippet']['topLevelComment']['snippet']['authorChannelId']
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            likes = item['snippet']['topLevelComment']['snippet']['likeCount']
            comment_df = comment_df.append(pd.Series([author,comment,likes],index=comment_df.columns), ignore_index = True)
 
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break
 
    return comment_df

os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

In [ ]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

In [20]:
keyword = input('Enter a keyword: ')
search_videos_by_keyword(service, q=keyword, part='id,snippet', eventType='completed', type='video')

comments = get_video_comments(service, part='snippet', videoId='h2RzmSAZ4Hc', textFormat='plainText')

Enter a keyword:  What Trump said is unprecedented ... and it's not even Friday


HttpError: <HttpError 403 when requesting https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=zc5RMAfg9mA&textFormat=plainText&alt=json returned "The video identified by the <code><a href="/youtube/v3/docs/commentThreads/list#videoId">videoId</a></code> parameter has disabled comments.">

In [38]:
def get_videos(service, **kwargs):
    final_results = []
    results = service.search().list(**kwargs).execute()
    i = 0
    max_pages = 3
    while results and i < max_pages:
        final_results.extend(results['items'])
 
        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.search().list(**kwargs).execute()
            i += 1
        else:
            break
 
    return final_results

def search_videos_by_keyword(service, **kwargs):
    results = get_videos(service, **kwargs)
    return results

    
test = search_videos_by_keyword(service, q='Neural Network Learns to Play Snake', part='id,snippet', eventType='completed', type='video')


In [40]:
test[1]

{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/V6iP4AII4AhqiAsNj6eAAUZ-Vgw"',
 'id': {'kind': 'youtube#video', 'videoId': 'Sui3BUyIie8'},
 'kind': 'youtube#searchResult',
 'snippet': {'channelId': 'UC-FWuMytGJQARZnviW6p8SQ',
  'channelTitle': 'Geek Code',
  'description': 'Live! Visit our website if you want to download !',
  'liveBroadcastContent': 'none',
  'publishedAt': '2017-06-24T00:36:55.000Z',
  'thumbnails': {'default': {'height': 90,
    'url': 'https://i.ytimg.com/vi/Sui3BUyIie8/default.jpg',
    'width': 120},
   'high': {'height': 360,
    'url': 'https://i.ytimg.com/vi/Sui3BUyIie8/hqdefault.jpg',
    'width': 480},
   'medium': {'height': 180,
    'url': 'https://i.ytimg.com/vi/Sui3BUyIie8/mqdefault.jpg',
    'width': 320}},
  'title': 'SNAKE NEURAL NETWORK LIVE TRAINING'}}

In [56]:
request = service.search().list(
    q='cnn',
    part='id,snippet',
    eventType='completed',
    type='video',
    maxResults='3'
    )

In [57]:
request.execute()

{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/8JECkdoEKHsBXhzh2GiDD2b3ha4"',
 'items': [{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/3JAr-H8sSO2RWhQl5fEyAsN-vsA"',
   'id': {'kind': 'youtube#video', 'videoId': 'gMCGqsO_bEg'},
   'kind': 'youtube#searchResult',
   'snippet': {'channelId': 'UCef1-8eOpJgud7szVPlZQAQ',
    'channelTitle': 'CNN-News18',
    'description': 'Watch all the current, latest and breaking news only on CNN NEWS18 live TV. The one-stop destination for live news on politics, entertainment, sports, gadgets, ...',
    'liveBroadcastContent': 'none',
    'publishedAt': '2019-08-17T13:21:18.000Z',
    'thumbnails': {'default': {'height': 90,
      'url': 'https://i.ytimg.com/vi/gMCGqsO_bEg/default.jpg',
      'width': 120},
     'high': {'height': 360,
      'url': 'https://i.ytimg.com/vi/gMCGqsO_bEg/hqdefault.jpg',
      'width': 480},
     'medium': {'height': 180,
      'url': 'https://i.ytimg.com/vi/gMCGqsO_bEg/mqdefault.jpg',
      'width': 320}},
    'title': 'CNN-News18 LIVE |

In [129]:
request = service.channels().list(
    forUsername='fox',
    part='id,snippet'
    ).execute()
request
#channel_id = request['items'][0]['id']

{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/kS6ybUO2AbMbY6PJYQiTBQ57J3I"',
 'items': [{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/NWJOgvqPJGfhoKmLhgM2TxVzqMk"',
   'id': 'UCUTuhx9d0GDlQ2Gj1BmhpfA',
   'kind': 'youtube#channel',
   'snippet': {'customUrl': 'Fox',
    'description': '',
    'localized': {'description': '', 'title': 'Fox'},
    'publishedAt': '2005-08-05T22:32:24.000Z',
    'thumbnails': {'default': {'height': 88,
      'url': 'https://yt3.ggpht.com/a/AGF-l78et7f0i6LhnpPpBE1sv2_86w68YS6J8QXVoQ=s88-c-k-c0xffffffff-no-rj-mo',
      'width': 88},
     'high': {'height': 800,
      'url': 'https://yt3.ggpht.com/a/AGF-l78et7f0i6LhnpPpBE1sv2_86w68YS6J8QXVoQ=s800-c-k-c0xffffffff-no-rj-mo',
      'width': 800},
     'medium': {'height': 240,
      'url': 'https://yt3.ggpht.com/a/AGF-l78et7f0i6LhnpPpBE1sv2_86w68YS6J8QXVoQ=s240-c-k-c0xffffffff-no-rj-mo',
      'width': 240}},
    'title': 'Fox'}}],
 'kind': 'youtube#channelListResponse',
 'pageInfo': {'resultsPerPage': 5, 'totalResults': 1}}

In [95]:
request = service.channels().list(
    id=channel_id,
    part='id,snippet'
    ).execute()
request

{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/jtdZ-2M7os2H2NN2kekFXiKw5y8"',
 'items': [{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/xc82qIk3HuZwLAjind51YguaMQI"',
   'id': 'UCupvZG-5ko_eiXAupbDfxWw',
   'kind': 'youtube#channel',
   'snippet': {'customUrl': 'CNN',
    'description': "CNN operates as a division of Turner Broadcasting System, which is a subsidiary of Warner Media. CNN identifies itself as -- and is widely known to be - the most trusted source for news and information. The CNN umbrella includes nine cable and satellite television networks, two radio networks, the CNN Digital Network, which is the top network of news Web sites in the United States, and CNN Newsource, the world's most extensively syndicated news service. CNN is proud of our ability to bring you up-to-the-minute news from around the world, as a result of our many extensions.",
    'localized': {'description': "CNN operates as a division of Turner Broadcasting System, which is a subsidiary of Warner Media. CNN identifies

In [106]:
request = service.search().list(
    q='Mayor de Blasio',
    channelId=channel_id,
    part='id,snippet',
    eventType='completed',
    type='video',
    maxResults='3'
    )

In [107]:
request.execute()

{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/thxDYb21jTlPmuKT-C8lfsSfAds"',
 'items': [{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/-OKdNHJxw7IarRJfPoFF2lEjbXA"',
   'id': {'kind': 'youtube#video', 'videoId': 'IYoSGv5WeW4'},
   'kind': 'youtube#searchResult',
   'snippet': {'channelId': 'UCupvZG-5ko_eiXAupbDfxWw',
    'channelTitle': 'CNN',
    'description': "Governor Cuomo, Mayor de Blasio and New York Police Commissioner O'Neill join families and other guest at 9/11 ceremony.",
    'liveBroadcastContent': 'none',
    'publishedAt': '2018-09-11T16:36:40.000Z',
    'thumbnails': {'default': {'height': 90,
      'url': 'https://i.ytimg.com/vi/IYoSGv5WeW4/default.jpg',
      'width': 120},
     'high': {'height': 360,
      'url': 'https://i.ytimg.com/vi/IYoSGv5WeW4/hqdefault.jpg',
      'width': 480},
     'medium': {'height': 180,
      'url': 'https://i.ytimg.com/vi/IYoSGv5WeW4/mqdefault.jpg',
      'width': 320}},
    'title': '9/11 memorial ceremony in New York - September 11th, 2018'}}],
 'ki

In [126]:
def get_video_comments(service, **kwargs):
    # results = service.commentThreads().list(
    #     part='snippet',
    #     videoId='h2RzmSAZ4Hc',
    #     textFormat='plainText'
    #     ).execute()
    # results
    comment_df = pd.DataFrame(columns=['author','comment','likes'])
    results = service.commentThreads().list(**kwargs).execute()
 
    while results:
        for item in results['items']:
            author = item['snippet']['topLevelComment']['snippet']['authorChannelId']
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            likes = item['snippet']['topLevelComment']['snippet']['likeCount']
            comment_df = comment_df.append(pd.Series([author,comment,likes],index=comment_df.columns), ignore_index = True)
 
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break
 
    return comment_df



comments = get_video_comments(service, part='snippet', videoId='h2RzmSAZ4Hc', textFormat='plainText')

In [127]:
comments

author  \
0    {'value': 'UCsKDUhOUUB4QEP31JMwkrkw'}   
1    {'value': 'UCoetlyPkUiA7nLC_IsLVQ_w'}   
2    {'value': 'UCpfBxOREvwwX65EqYXRrLTQ'}   
3    {'value': 'UCoetlyPkUiA7nLC_IsLVQ_w'}   
4    {'value': 'UCh0wrWmkVP0XFOmldSEX77w'}   
..                                     ...   
370  {'value': 'UC4hxNaN_FqjXslqnjvjZxoA'}   
371  {'value': 'UCQD0yGciGUW_DUJGO6VFtBw'}   
372  {'value': 'UCKi71kdGB-9Wl5_EEDTQCsg'}   
373  {'value': 'UCZ880zph14jDpY_d82NmT3g'}   
374  {'value': 'UCLe9VXOeo18Yw9EsmlIwAyw'}   

                                               comment likes  
0              RAPIST!!! Don Lemon needs to be FIRED!!     0  
1               Cnn you should be ashamed of yourselfs     1  
2    I don't know who this dude is but I like him b...     0  
3       What is this guy saying? Me not understanding.     1  
4                     I bet Fareed smells really nice!     0  
..                                                 ...   ...  
370                                          Early boi     0  
371  I thought Trump said he would destroy ISIS in ...     9  
372                one word LEAVE!!! problem solved!!!     6  
373                            Trump 2020 🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸     8  
374                    Tariffs on China are justified.     7  

[375 rows x 3 columns]

In [113]:
results = service.commentThreads().list(
    part='snippet',
    videoId='h2RzmSAZ4Hc',
    textFormat='plainText'
    ).execute()
results

{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/ciUnW7gNmzcW7KNoZfyxmnEkISo"',
 'items': [{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/VOHRy3MsaBneuWHuv60xZW_jruM"',
   'id': 'UgweNKD7Wv3vLejTyjN4AaABAg',
   'kind': 'youtube#commentThread',
   'snippet': {'canReply': True,
    'isPublic': True,
    'topLevelComment': {'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/SY3T3_0L96yzoWTYNQIDQqNgipw"',
     'id': 'UgweNKD7Wv3vLejTyjN4AaABAg',
     'kind': 'youtube#comment',
     'snippet': {'authorChannelId': {'value': 'UCjdqUPMT-LWx0ftpEosrDTA'},
      'authorChannelUrl': 'http://www.youtube.com/channel/UCjdqUPMT-LWx0ftpEosrDTA',
      'authorDisplayName': 'T Dub',
      'authorProfileImageUrl': 'https://yt3.ggpht.com/--DWVDBdatSY/AAAAAAAAAAI/AAAAAAAAAAA/jARx7Xkb2kk/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
      'canRate': True,
      'likeCount': 0,
      'publishedAt': '2019-08-18T22:44:48.000Z',
      'textDisplay': "Hi. I work for CNN and i'm going to fill your soft, leftist brain full of bullshit because I want y

In [121]:
comment_df = pd.DataFrame(columns=['author','comment','likes'])

c = 'comment'
like = 'likes'
autr = 'authorsss'
comment_df.append(pd.Series([c,like,autr],index=comment_df.columns), ignore_index = True)


author comment      likes
0  comment   likes  authorsss

In [133]:
def search_channel(service, search_term):
    request = service.search().list(
        q=search_term,
        part='id,snippet',
        type='channel',
        maxResults='20'
    ).execute()
    
    channel_df = pd.DataFrame(columns=['channel_id','channel'])
    for results in request:
        chan = request['items'][0]['channelTitle']
        c_id = request['items'][0]['channelId']
        channel_df = channel_df.append(pd.Series([chan, c_id],index=channel_df.columns), ignore_index = True)
    
    return channel_df
        


def find_channel_id(service, search_term):
    request = service.channels().list(
    forUsername=search_term,
    part='id,snippet'
    ).execute()
    channel_id = request['items'][0]['id']
    return channel_id


comment_df = pd.DataFrame(columns=['author','comment','likes'])
    results = service.commentThreads().list(**kwargs).execute()
 
    while results:
        for item in results['items']:
            author = item['snippet']['topLevelComment']['snippet']['authorChannelId']
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            likes = item['snippet']['topLevelComment']['snippet']['likeCount']
            comment_df = comment_df.append(pd.Series([author,comment,likes],index=comment_df.columns), ignore_index = True)

request = service.search().list(
    q='cnn',
    part='id,snippet',
    type='channel',
    maxResults='20'
    )
request.execute()

{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/2S6eo54pG9JE-sVJqQDFyabAO5E"',
 'items': [{'etag': '"0UM_wBUsFuT6ekiIlwaHvyqc80M/f10hjuC3bLJD8gUed_U6_N0H5L0"',
   'id': {'channelId': 'UCupvZG-5ko_eiXAupbDfxWw', 'kind': 'youtube#channel'},
   'kind': 'youtube#searchResult',
   'snippet': {'channelId': 'UCupvZG-5ko_eiXAupbDfxWw',
    'channelTitle': 'CNN',
    'description': 'CNN operates as a division of Turner Broadcasting System, which is a subsidiary of Warner Media. CNN identifies itself as -- and is widely known to be - the ...',
    'liveBroadcastContent': 'none',
    'publishedAt': '2005-10-02T16:06:36.000Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/-K12xTWC-rMI/AAAAAAAAAAI/AAAAAAAAAAA/2N_u5pcKB3w/s88-c-k-no-mo-rj-c0xffffff/photo.jpg'},
     'high': {'url': 'https://yt3.ggpht.com/-K12xTWC-rMI/AAAAAAAAAAI/AAAAAAAAAAA/2N_u5pcKB3w/s800-c-k-no-mo-rj-c0xffffff/photo.jpg'},
     'medium': {'url': 'https://yt3.ggpht.com/-K12xTWC-rMI/AAAAAAAAAAI/AAAAAAAAAAA/2N_u5pcKB3w/s240-c-

In [134]:
comments.tail()

author  \
370  {'value': 'UC4hxNaN_FqjXslqnjvjZxoA'}   
371  {'value': 'UCQD0yGciGUW_DUJGO6VFtBw'}   
372  {'value': 'UCKi71kdGB-9Wl5_EEDTQCsg'}   
373  {'value': 'UCZ880zph14jDpY_d82NmT3g'}   
374  {'value': 'UCLe9VXOeo18Yw9EsmlIwAyw'}   

                                               comment likes  
370                                          Early boi     0  
371  I thought Trump said he would destroy ISIS in ...     9  
372                one word LEAVE!!! problem solved!!!     6  
373                            Trump 2020 🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸     8  
374                    Tariffs on China are justified.     7

In [135]:
import spacy

import numpy as np
from datetime import datetime, timedelta

#nlp = spacy.load('en')
nlp = spacy.load('en_core_web_lg')

In [ ]:
from scipy import spatial
from nltk.chunk import conlltags2tree
from spacy import displacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
nlp = spacy.load('en_core_web_lg')

cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)
 
man = nlp.vocab['man'].vector
woman = nlp.vocab['woman'].vector
queen = nlp.vocab['queen'].vector
king = nlp.vocab['king'].vector
 
# We now need to find the closest vector in the vocabulary to the result of "man" - "woman" + "queen"
maybe_king = man - woman + queen
computed_similarities = []
 
for word in nlp.vocab:
    # Ignore words without vectors
    if not word.has_vector:
        continue
 
    similarity = cosine_similarity(maybe_king, word.vector)
    computed_similarities.append((word, similarity))
 
computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])
print([w[0].text for w in computed_similarities[:10]])

sentiment_analyzer = SentimentIntensityAnalyzer()
def polarity_scores(doc):
    return sentiment_analyzer.polarity_scores(doc.text)
 
Doc.set_extension('polarity_scores', getter=polarity_scores)
 
nlp = spacy.load('en')
doc = nlp("Really Whaaat event apple nice! it!")
print(doc._.polarity_scores)

In [145]:
from spacy.tokens import Doc
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiment_analyzer = SentimentIntensityAnalyzer()
def polarity_scores(doc):
    return sentiment_analyzer.polarity_scores(doc.text)
 
Doc.set_extension('polarity_scores', getter=polarity_scores)
 
nlp = spacy.load('en')
doc = nlp("Really Whaaat event apple nice! it!")
print(doc._.polarity_scores)

{'neg': 0.0, 'neu': 0.596, 'pos': 0.404, 'compound': 0.5242}


In [180]:
comment = nlp(comments.loc[18]['comment'])
print(comment)
#print(doc._.polarity_scores)
for sent in comment.sents:
    for token in sent:
        subjects = []
        if token.dep_ == 'nsubj':
            subjects.append(token.text)  
        if token.dep_ == 'amod':
            print(token.text)
    print(sent)
    print(polarity_scores(sent))
    #print()
# [sent.text for sent in doc.sents]
# [ent.text for ent in doc.ents]
#displacy.render(doc, style="dep", jupyter=True)

No, we need to gtfo. Let them sort out their own problems, even if it takes decades. We can't keep intervening forever. We can't keep changing regimes forever. Same in Africa and South America. Take refugees when necessary, but otherwise, our nation has problems inside we need to take care of
No, we need to gtfo.
{'neg': 0.355, 'neu': 0.645, 'pos': 0.0, 'compound': -0.296}
own
Let them sort out their own problems, even if it takes decades.
{'neg': 0.197, 'neu': 0.803, 'pos': 0.0, 'compound': -0.4019}
We can't keep intervening forever.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
We can't keep changing regimes forever.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Same in Africa and South America.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Take refugees when necessary, but otherwise, our nation has problems inside we need to take care of
{'neg': 0.155, 'neu': 0.656, 'pos': 0.188, 'compound': 0.1901}


In [176]:
comment = nlp(comments.loc[18]['comment'])
for chunk in comment.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)
    
for token in comment:
    print(token.text, token.dep_)

we we nsubj need
them them nsubj sort
their own problems problems dobj sort
it it nsubj takes
decades decades dobj takes
We We nsubj keep
We We nsubj keep
regimes regimes dobj changing
Africa Africa pobj in
South America America conj Africa
refugees refugees dobj Take
our nation nation nsubj has
problems problems dobj has
we we nsubj need
care care dobj take
No intj
, punct
we nsubj
need ROOT
to prep
gtfo pobj
. punct
Let ROOT
them nsubj
sort ccomp
out prt
their poss
own amod
problems dobj
, punct
even advmod
if mark
it nsubj
takes advcl
decades dobj
. punct
We nsubj
ca aux
n't neg
keep ROOT
intervening xcomp
forever advmod
. punct
We nsubj
ca aux
n't neg
keep ROOT
changing xcomp
regimes dobj
forever advmod
. punct
Same ROOT
in prep
Africa pobj
and cc
South compound
America conj
. punct
Take ROOT
refugees dobj
when advmod
necessary advcl
, punct
but cc
otherwise advmod
, punct
our poss
nation nsubj
has conj
problems dobj
inside advmod
we nsubj
need relcl
to aux
take xcomp
care dobj
of 

TypeError: serve() got an unexpected keyword argument 'jupyter'